In [221]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import copy

df_hospital=pd.read_csv(r'../data/hospital_resources/Hospital_resources_hanmin.csv', encoding='ISO-8859-1')

In [220]:
ven_aloc_solu = np.load(r'../data/hospital_resources/Transported_ventilators.npy', allow_pickle=True)
up_tri = np.triu(ven_aloc_solu, 1)
lw_tri = np.tril(ven_aloc_solu, -1)
full_dec_mat = lw_tri-up_tri.transpose()  # use to decide who is sender who is receiver
sender_index = np.nonzero(ven_aloc_solu.sum(axis=1)>0)
receiver_index = np.nonzero(ven_aloc_solu.sum(axis=0)>0)
neutral_index = np.where(full_dec_mat.sum(axis=1) == 0)

sen_hosp = df_hospital.copy()
rec_hosp = df_hospital.copy()
neu_hosp = df_hospital.copy()

sen_hosp = sen_hosp.iloc[sender_index]
rec_hosp = rec_hosp.iloc[receiver_index]
neu_hosp = neu_hosp.iloc[neutral_index]


print('plotting hospitals ...')

fig = go.Figure()
fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = df_hospital['Longitude'],
    lat = df_hospital['Latitude'],
    hoverinfo = 'text',
    text = df_hospital['Inst'],
    mode = 'markers',
    marker = dict(
        size = 5,
        color = 'black',
    )
    ))
fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = sen_hosp['Longitude'],
    lat = sen_hosp['Latitude'],
    hoverinfo = 'text',
    text = sen_hosp['Inst'],
    mode = 'markers',
    marker = dict(
        size = 5,
        color = 'green',
    )
    ))

fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = rec_hosp['Longitude'],
    lat = rec_hosp['Latitude'],
    hoverinfo = 'text',
    text = rec_hosp['Inst'],
    mode = 'markers',
    marker = dict(
        size = 5,
        color = 'red',
    )
    ))

non_zero_ind = np.nonzero(ven_aloc_solu)
sen_ind = non_zero_ind[0]
rec_ind = non_zero_ind[1]

data = {'orig_lat': [-1.1]*len(rec_ind), 
        'orig_lon': [-1.1]*len(rec_ind), 
        'dest_lat': [-1.1]*len(rec_ind),
        'dest_lon': [-1.1]*len(rec_ind),
        'orig_hosp': ['']*len(rec_ind),
        'dest_hosp': ['']*len(rec_ind),
        'num_ven_tansp': [0]*len(rec_ind)} 
df_ven_reloc = pd.DataFrame(data)
ind = 0
for orig_ind, dest_ind in zip(sen_ind,rec_ind):
        df_ven_reloc['orig_lat'][ind] = df_hospital['Latitude'][orig_ind]
        df_ven_reloc['orig_lon'][ind] = df_hospital['Longitude'][orig_ind]
        df_ven_reloc['orig_hosp'][ind] = df_hospital['Inst'][orig_ind]  # from hospital
        
        df_ven_reloc['dest_lat'][ind] = df_hospital['Latitude'][dest_ind]
        df_ven_reloc['dest_lon'][ind] = df_hospital['Longitude'][dest_ind]
        df_ven_reloc['dest_hosp'][ind] = df_hospital['Inst'][dest_ind]  # to hospital
        df_ven_reloc['num_ven_tansp'][ind] = ven_aloc_solu[orig_ind, dest_ind]
        ind += 1

print('saving to csv ...')

df_ven_reloc.to_csv('../data/hospital_resources/df_ven_reloc.csv')

print('plotting reallocation ...')

for i in range(len(df_ven_reloc)):
    fig.add_trace(
        go.Scattergeo(
            locationmode = 'USA-states',
            lon = [df_ven_reloc['orig_lon'][i], df_ven_reloc['dest_lon'][i]],
            lat = [df_ven_reloc['orig_lat'][i], df_ven_reloc['dest_lat'][i]],
            mode = 'lines',
            line = dict(width = 1.5,color = 'blue'),
            opacity = float(df_ven_reloc['num_ven_tansp'][i]) / float(df_ven_reloc['num_ven_tansp'].max()),
        )
    )

fig.update_layout(
    title_text = 'Ventilator reallocation results',
    showlegend = False,
    geo = dict(
        resolution = 50, 
        scope = 'europe',
#         center = {'lat': (df_ven_reloc['dest_lat'].min() + df_ven_reloc['dest_lat'].max())/2,
#                   'lon': (df_ven_reloc['orig_lon'].min() + df_ven_reloc['orig_lon'].max())/2},
        projection_type = 'stereographic',
#         showland = True,
        showlakes = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
    ),
)

fig.show()

plotting hospitals ...
saving to csv ...
plotting reallocation ...


C:\Users\caha\PycharmProjects\ch_vs_virus_416\venv\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\caha\PycharmProjects\ch_vs_virus_416\venv\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\caha\PycharmProjects\ch_vs_virus_416\venv\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\caha\Py

In [222]:
ven_aloc_solu = np.load(r'../data/hospital_resources/Transported_ventilators_X1demand.npy', allow_pickle=True)
up_tri = np.triu(ven_aloc_solu, 1)
lw_tri = np.tril(ven_aloc_solu, -1)
full_dec_mat = lw_tri-up_tri.transpose()  # use to decide who is sender who is receiver
sender_index = np.nonzero(ven_aloc_solu.sum(axis=1)>0)
receiver_index = np.nonzero(ven_aloc_solu.sum(axis=0)>0)
neutral_index = np.where(full_dec_mat.sum(axis=1) == 0)

sen_hosp = df_hospital.copy()
rec_hosp = df_hospital.copy()
neu_hosp = df_hospital.copy()

sen_hosp = sen_hosp.iloc[sender_index]
rec_hosp = rec_hosp.iloc[receiver_index]
neu_hosp = neu_hosp.iloc[neutral_index]


print('plotting hospitals ...')

fig = go.Figure()
fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = df_hospital['Longitude'],
    lat = df_hospital['Latitude'],
    hoverinfo = 'text',
    text = df_hospital['Inst'],
    mode = 'markers',
    marker = dict(
        size = 5,
        color = 'black',
    )
    ))
fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = sen_hosp['Longitude'],
    lat = sen_hosp['Latitude'],
    hoverinfo = 'text',
    text = sen_hosp['Inst'],
    mode = 'markers',
    marker = dict(
        size = 5,
        color = 'green',
    )
    ))

fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = rec_hosp['Longitude'],
    lat = rec_hosp['Latitude'],
    hoverinfo = 'text',
    text = rec_hosp['Inst'],
    mode = 'markers',
    marker = dict(
        size = 5,
        color = 'red',
    )
    ))

non_zero_ind = np.nonzero(ven_aloc_solu)
sen_ind = non_zero_ind[0]
rec_ind = non_zero_ind[1]

data = {'orig_lat': [-1.1]*len(rec_ind), 
        'orig_lon': [-1.1]*len(rec_ind), 
        'dest_lat': [-1.1]*len(rec_ind),
        'dest_lon': [-1.1]*len(rec_ind),
        'orig_hosp': ['']*len(rec_ind),
        'dest_hosp': ['']*len(rec_ind),
        'num_ven_tansp': [0]*len(rec_ind)} 
df_ven_reloc = pd.DataFrame(data)
ind = 0
for orig_ind, dest_ind in zip(sen_ind,rec_ind):
        df_ven_reloc['orig_lat'][ind] = df_hospital['Latitude'][orig_ind]
        df_ven_reloc['orig_lon'][ind] = df_hospital['Longitude'][orig_ind]
        df_ven_reloc['orig_hosp'][ind] = df_hospital['Inst'][orig_ind]  # from hospital
        
        df_ven_reloc['dest_lat'][ind] = df_hospital['Latitude'][dest_ind]
        df_ven_reloc['dest_lon'][ind] = df_hospital['Longitude'][dest_ind]
        df_ven_reloc['dest_hosp'][ind] = df_hospital['Inst'][dest_ind]  # to hospital
        df_ven_reloc['num_ven_tansp'][ind] = ven_aloc_solu[orig_ind, dest_ind]
        ind += 1

print('saving to csv ...')

df_ven_reloc.to_csv('../data/hospital_resources/df_ven_reloc_X1demand.csv')

print('plotting reallocation ...')

for i in range(len(df_ven_reloc)):
    fig.add_trace(
        go.Scattergeo(
            locationmode = 'USA-states',
            lon = [df_ven_reloc['orig_lon'][i], df_ven_reloc['dest_lon'][i]],
            lat = [df_ven_reloc['orig_lat'][i], df_ven_reloc['dest_lat'][i]],
            mode = 'lines',
            line = dict(width = 1.5,color = 'blue'),
            opacity = float(df_ven_reloc['num_ven_tansp'][i]) / float(df_ven_reloc['num_ven_tansp'].max()),
        )
    )

fig.update_layout(
    title_text = 'Ventilator reallocation results',
    showlegend = False,
    geo = dict(
        resolution = 50, 
        scope = 'europe',
#         center = {'lat': (df_ven_reloc['dest_lat'].min() + df_ven_reloc['dest_lat'].max())/2,
#                   'lon': (df_ven_reloc['orig_lon'].min() + df_ven_reloc['orig_lon'].max())/2},
        projection_type = 'stereographic',
#         showland = True,
        showlakes = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
    ),
)

fig.show()

plotting hospitals ...
saving to csv ...
plotting reallocation ...


C:\Users\caha\PycharmProjects\ch_vs_virus_416\venv\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\caha\PycharmProjects\ch_vs_virus_416\venv\lib\site-packages\ipykernel_launcher.py:74: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\caha\PycharmProjects\ch_vs_virus_416\venv\lib\site-packages\ipykernel_launcher.py:75: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\caha\Py

In [223]:
ven_aloc_solu = np.load(r'../data/hospital_resources/Transported_ventilators_X2demand.npy', allow_pickle=True)
up_tri = np.triu(ven_aloc_solu, 1)
lw_tri = np.tril(ven_aloc_solu, -1)
full_dec_mat = lw_tri-up_tri.transpose()  # use to decide who is sender who is receiver
sender_index = np.nonzero(ven_aloc_solu.sum(axis=1)>0)
receiver_index = np.nonzero(ven_aloc_solu.sum(axis=0)>0)
neutral_index = np.where(full_dec_mat.sum(axis=1) == 0)

sen_hosp = df_hospital.copy()
rec_hosp = df_hospital.copy()
neu_hosp = df_hospital.copy()

sen_hosp = sen_hosp.iloc[sender_index]
rec_hosp = rec_hosp.iloc[receiver_index]
neu_hosp = neu_hosp.iloc[neutral_index]


print('plotting hospitals ...')

fig = go.Figure()
fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = df_hospital['Longitude'],
    lat = df_hospital['Latitude'],
    hoverinfo = 'text',
    text = df_hospital['Inst'],
    mode = 'markers',
    marker = dict(
        size = 5,
        color = 'black',
    )
    ))
fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = sen_hosp['Longitude'],
    lat = sen_hosp['Latitude'],
    hoverinfo = 'text',
    text = sen_hosp['Inst'],
    mode = 'markers',
    marker = dict(
        size = 5,
        color = 'green',
    )
    ))

fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = rec_hosp['Longitude'],
    lat = rec_hosp['Latitude'],
    hoverinfo = 'text',
    text = rec_hosp['Inst'],
    mode = 'markers',
    marker = dict(
        size = 5,
        color = 'red',
    )
    ))

non_zero_ind = np.nonzero(ven_aloc_solu)
sen_ind = non_zero_ind[0]
rec_ind = non_zero_ind[1]

data = {'orig_lat': [-1.1]*len(rec_ind), 
        'orig_lon': [-1.1]*len(rec_ind), 
        'dest_lat': [-1.1]*len(rec_ind),
        'dest_lon': [-1.1]*len(rec_ind),
        'orig_hosp': ['']*len(rec_ind),
        'dest_hosp': ['']*len(rec_ind),
        'num_ven_tansp': [0]*len(rec_ind)} 
df_ven_reloc = pd.DataFrame(data)
ind = 0
for orig_ind, dest_ind in zip(sen_ind,rec_ind):
        df_ven_reloc['orig_lat'][ind] = df_hospital['Latitude'][orig_ind]
        df_ven_reloc['orig_lon'][ind] = df_hospital['Longitude'][orig_ind]
        df_ven_reloc['orig_hosp'][ind] = df_hospital['Inst'][orig_ind]  # from hospital
        
        df_ven_reloc['dest_lat'][ind] = df_hospital['Latitude'][dest_ind]
        df_ven_reloc['dest_lon'][ind] = df_hospital['Longitude'][dest_ind]
        df_ven_reloc['dest_hosp'][ind] = df_hospital['Inst'][dest_ind]  # to hospital
        df_ven_reloc['num_ven_tansp'][ind] = ven_aloc_solu[orig_ind, dest_ind]
        ind += 1

print('saving to csv ...')

df_ven_reloc.to_csv('../data/hospital_resources/df_ven_reloc_X2demand.csv')

print('plotting reallocation ...')

for i in range(len(df_ven_reloc)):
    fig.add_trace(
        go.Scattergeo(
            locationmode = 'USA-states',
            lon = [df_ven_reloc['orig_lon'][i], df_ven_reloc['dest_lon'][i]],
            lat = [df_ven_reloc['orig_lat'][i], df_ven_reloc['dest_lat'][i]],
            mode = 'lines',
            line = dict(width = 1.5,color = 'blue'),
            opacity = float(df_ven_reloc['num_ven_tansp'][i]) / float(df_ven_reloc['num_ven_tansp'].max()),
        )
    )

fig.update_layout(
    title_text = 'Ventilator reallocation results',
    showlegend = False,
    geo = dict(
        resolution = 50, 
        scope = 'europe',
#         center = {'lat': (df_ven_reloc['dest_lat'].min() + df_ven_reloc['dest_lat'].max())/2,
#                   'lon': (df_ven_reloc['orig_lon'].min() + df_ven_reloc['orig_lon'].max())/2},
        projection_type = 'stereographic',
#         showland = True,
        showlakes = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
    ),
)

fig.show()

plotting hospitals ...
saving to csv ...
plotting reallocation ...


C:\Users\caha\PycharmProjects\ch_vs_virus_416\venv\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\caha\PycharmProjects\ch_vs_virus_416\venv\lib\site-packages\ipykernel_launcher.py:74: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\caha\PycharmProjects\ch_vs_virus_416\venv\lib\site-packages\ipykernel_launcher.py:75: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\caha\Py